In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import pytesseract
from PIL import Image

class TextDetectionOCR:
    def __init__(self, yolo_model_path):
        """
        Khởi tạo mô hình với đường dẫn đến file weights của YOLO
        """
        # Tải mô hình YOLO
        self.model = YOLO(yolo_model_path)
        
        # Cấu hình các tham số
        self.conf_threshold = 0.25
        self.iou_threshold = 0.45
        
    def preprocess_image(self, image):
        """
        Tiền xử lý ảnh đầu vào
        """
        # Chuyển sang ảnh xám
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Khử nhiễu
        denoised = cv2.fastNlMeansDenoising(gray)
        
        # Tăng độ tương phản
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        contrast = clahe.apply(denoised)
        
        return contrast

    def detect_text_regions(self, image):
        """
        Sử dụng YOLO để phát hiện vùng chứa text
        """
        # Thực hiện dự đoán với YOLO
        results = self.model(image, conf=self.conf_threshold, iou=self.iou_threshold)
        
        # Lấy các bounding box
        boxes = []
        for result in results:
            boxes.extend(result.boxes.xyxy.cpu().numpy())
            
        return np.array(boxes)

    def extract_text_from_region(self, image, box, lang='vie'):
        """
        Trích xuất text từ một vùng ảnh cụ thể
        """
        # Cắt vùng ảnh theo bounding box
        x1, y1, x2, y2 = map(int, box)
        region = image[y1:y2, x1:x2]
        
        # Tiền xử lý vùng ảnh
        processed_region = self.preprocess_image(region)
        
        # Chuyển sang định dạng PIL
        pil_image = Image.fromarray(processed_region)
        
        # Trích xuất text
        text = pytesseract.image_to_string(pil_image, lang=lang)
        
        return text.strip()

    def process_image(self, image_path, lang='vie', visualize=False):
        """
        Xử lý ảnh và trích xuất text từ tất cả các vùng được phát hiện
        """
        # Đọc ảnh
        image = cv2.imread(image_path)
        original = image.copy()
        
        # Phát hiện vùng text
        boxes = self.detect_text_regions(image)
        
        results = []
        # Xử lý từng vùng
        for box in boxes:
            text = self.extract_text_from_region(image, box, lang)
            if text:
                results.append({
                    'text': text,
                    'box': box.tolist()
                })
                
        # Vẽ kết quả nếu yêu cầu
        if visualize:
            for result in results:
                box = result['box']
                cv2.rectangle(original, 
                            (int(box[0]), int(box[1])), 
                            (int(box[2]), int(box[3])), 
                            (0, 255, 0), 2)
                cv2.putText(original, 
                           result['text'][:20] + "...", 
                           (int(box[0]), int(box[1] - 10)),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            return results, original
            
        return results


In [2]:
detector = TextDetectionOCR('yolov8s.pt')

# Xử lý ảnh
results, visualized_image = detector.process_image(
    './data/sample/ocr.png',
    visualize=True
)

# In kết quả
print("Kết quả nhận diện:")
for result in results:
    print(f"Text: {result['text']}")
    print(f"Vị trí: {result['box']}\n")

# Hiển thị ảnh kết quả
# cv2.imshow('Results', visualized_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Lưu ảnh kết quả
cv2.imwrite('result.jpg', visualized_image)


0: 160x640 1 train, 49.6ms
Speed: 6.2ms preprocess, 49.6ms inference, 614.8ms postprocess per image at shape (1, 3, 160, 640)
Kết quả nhận diện:


True

In [3]:
!pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00


In [4]:
import easyocr

# Load the image file path
image_path = './data/sample/ocr.png'  # Replace with your local path

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'], gpu =  True)

# Perform text detection and recognition
results = reader.readtext(image_path)

# Print the results
for (bbox, text, confidence) in results:
    print(f"Detected text: '{text}' with confidence: {confidence}")

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteDetected text: 'CHE' with confidence: 0.9995971751298354
Detected text: '1-01' with confidence: 0.6182919604570368
Detected text: 'BUR' with confidence: 0.9999307618526357
Detected text: '49.55' with confidence: 0.8905346903352951


/home/thiendc/projects/.thienenv/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


In [5]:
results

[([[95, 15], [129, 15], [129, 33], [95, 33]], 'CHE', 0.9995971751298354),
 ([[139, 15], [183, 15], [183, 35], [139, 35]], '1-01', 0.6182919604570368),
 ([[191, 15], [223, 15], [223, 33], [191, 33]], 'BUR', 0.9999307618526357),
 ([[99, 39], [139, 39], [139, 55], [99, 55]], '49.55', 0.8905346903352951)]